# Consolidation of wealth

Let's consider a game in which all contestants get 100 at a start. Then they play the following game: <br>
<ul>
    <li> for every round players are randomly divided to pairs</li>
    <li> each pair tosses a coin 'n_rounds' times. </li>
    <li> for every heads 10 from person A gets transfered to person B </li>
    <li> for every tails 10 gets transfered from person B to person A </li>
</ul>

In [219]:
import random
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use("ggplot")

In [15]:
contestants = {"contestant_" + str(i): 100 for i in range (1, 101)}
brokes = {}

In [3]:
n_games = 10
n_rounds = 5

In [245]:
class Contestant(object):
    def __init__(self, name, money=100, increment=10):
        self._name = name
        self._money = money
        self.increment = increment
        
    def get_money(self):
        return self._money
    
    def get_name(self):
        return self._name
    
    def is_broke(self):
        if self._money < 0:
            raise ValueError
        elif self._money == 0:
            return True
        else:
            return False
    
    def increase_money(self):
        self._money += self.increment
        
    def decrease_money(self):
        self._money -= self.increment

In [252]:
class Game(object):
    def __init__(self, n_rounds, n_plays_in_round, contestants):
        self.n_rounds = n_rounds
        self.n_plays_in_round = n_plays_in_round
        self.contestants = contestants
        self.brokes = {}
        self.n_g = 0
        
    def play_game_for_pair(self, contestant_1, contestant_2):
        
        for _ in range(self.n_plays_in_round):
            
            if random.random() > 0.5:
                contestant_1.increase_money()
                contestant_2.decrease_money()
            else:
                contestant_2.increase_money()
                contestant_1.decrease_money()
            
            broke = False
            if contestant_1.is_broke():
                self.make_broke(contestant_1.get_name())
                broke = True
            elif contestant_2.is_broke():
                self.make_broke(contestant_2.get_name())
                broke = True
            else:
                self.n_g += 1
                
            if broke:
                break
            
            
                
    def make_broke(self, contestant_name):
        print(len(self.contestants), '------', len(self.brokes))
        print(self.n_g)
        name = contestant_name
        try:
            self.brokes[contestant_name] = self.contestants[contestant_name]
        
            del self.contestants[contestant_name]
        except:
            print(contestant_name)
            print("in contestants:", contestant_name in self.contestants)
            print(contestant_name in self.brokes)
        print('fini')
        
    def play_game(self):
        
        players = np.random.permutation(list(self.contestants.values()))

        if len(self.contestants) < 2:
            raise ValueError
        for _ in range(self.n_rounds):
            for c1, c2 in zip(players[::2], players[1::2]):
                self.play_game_for_pair(c1, c2)
    
    def play_round(self):
        
        players = np.random.permutation(list(self.contestants.values()))
        if len(self.contestants) < 2:
            raise ValueError
        for c1, c2 in zip(players[::2], players[1::2]):
            self.play_game_for_pair(c1, c2)
                
                
    def get_results(self):
        return [cont.get_money() for cont in self.contestants.values()]

In [253]:
g1 = Game(10, 10, {"contestant_" + str(i): Contestant("contestant_" + str(i)) for i in range(20)})

In [255]:
g1.contestants

{'contestant_0': <__main__.Contestant at 0x27c9e059390>,
 'contestant_1': <__main__.Contestant at 0x27c9e059898>,
 'contestant_10': <__main__.Contestant at 0x27c9ccee588>,
 'contestant_11': <__main__.Contestant at 0x27c9cd05080>,
 'contestant_12': <__main__.Contestant at 0x27c9cd050b8>,
 'contestant_14': <__main__.Contestant at 0x27c9cd05be0>,
 'contestant_15': <__main__.Contestant at 0x27c9cd05ac8>,
 'contestant_17': <__main__.Contestant at 0x27c9cd05ba8>,
 'contestant_18': <__main__.Contestant at 0x27c9cd05cc0>,
 'contestant_19': <__main__.Contestant at 0x27c9cd05c50>,
 'contestant_2': <__main__.Contestant at 0x27c9f4f7278>,
 'contestant_3': <__main__.Contestant at 0x27c9f4f7f98>,
 'contestant_5': <__main__.Contestant at 0x27c9e419e10>,
 'contestant_6': <__main__.Contestant at 0x27c9e284400>,
 'contestant_7': <__main__.Contestant at 0x27c9ccee5f8>,
 'contestant_8': <__main__.Contestant at 0x27c9ccee668>,
 'contestant_9': <__main__.Contestant at 0x27c9ccee4a8>}

In [256]:
for guy in g1.contestants.values():
    print(guy.get_money())

120
210
80
80
60
60
80
140
140
200
80
120
60
160
120
120
140


In [257]:
for guy in g1.brokes.values():
    print(guy.get_money())

-10
40
0
